# Установка необходимых библиотек

Перед началом работы убедимся, что у нас установлены все пакеты для:
- работы с PyTorch и torchaudio для загрузки и препроцессинга аудио,
- transformers для модели CLAP,
- librosa (при необходимости дополнительных фич),
- FAISS для быстрого поиска векторов,
- tqdm для наглядного прогресса в циклах.



In [1]:
!pip install torch torchaudio transformers librosa faiss-cpu tqdm


Defaulting to user installation because normal site-packages is not writeable


# Проверка доступности GPU

Хотим убедиться, что мы действительно работаем на GPU (если есть),  
чтобы ускорить вычисления эмбеддингов аудио.


In [2]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


GPU available: True
Device: NVIDIA GeForce GTX 1070


# Сбор списка mp3-файлов из каталога FMA Large

Проходим по всем поддиректориям, собираем пути к `.mp3`.  
В дальнейшем по этому списку будем итерировать и строить эмбеддинги.


In [3]:
import os

# Change this if needed
AUDIO_FOLDER = "/home/ivan/PycharmProjects/MPr/audio_samples/fma_large/fma_large"

mp3_files = []
for root, _, files in os.walk(AUDIO_FOLDER):
    for file in files:
        if file.endswith(".mp3"):
            mp3_files.append(os.path.join(root, file))

# Optional: limit to the first 300 for now
mp3_files = mp3_files

print(f"Loaded {len(mp3_files)} files.")


Loaded 106574 files.


# Загрузка модели CLAP

Используем предобученную модель `laion/clap-htsat-unfused`:
- `ClapProcessor` готовит аудио (и текст, если понадобится) в формат для модели.
- `ClapModel` выдаёт 512-мерный эмбеддинг аудио.


In [4]:
import torch
from transformers import ClapProcessor, ClapModel

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
model = ClapModel.from_pretrained("laion/clap-htsat-unfused").to(device)


# Функция `embed_audio`

1. Загрузить .mp3 через `torchaudio.load`.
2. Привести дорожку к моно, ресемплировать к 48 кГц.
3. Обрезать или дополнить до ровно 30 секунд.
4. Передать сигнал в CLAP и получить вектор признаков.
5. Обрабатывать ошибки (если файл битый).


# Главный loop по всем файлам

- Для каждого mp3 вызываем `embed_audio`.
- Накопленные эмбеддинги и пути сохраняем каждые N треков,  
  чтобы не потерять результат при сбое.
- По завершении сохраняем остаток и список неудач.


In [6]:
import os
import torch
import torchaudio
import numpy as np
from tqdm import tqdm

SAVE_EVERY = 1000  # Save checkpoint every N files
output_dir = "/home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large"
os.makedirs(output_dir, exist_ok=True)

def embed_audio(filepath):
    try:
        waveform, sr = torchaudio.load(filepath)
        waveform = waveform.mean(dim=0).unsqueeze(0)  # Mono

        if sr != 48000:
            waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=48000)(waveform)

        max_len = 48000 * 30  # 30s
        if waveform.shape[1] > max_len:
            waveform = waveform[:, :max_len]
        elif waveform.shape[1] < max_len:
            pad_len = max_len - waveform.shape[1]
            waveform = torch.nn.functional.pad(waveform, (0, pad_len))

        inputs = processor(
            audios=waveform.squeeze(0).cpu().numpy(),
            sampling_rate=48000,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            embedding = model.get_audio_features(**inputs)

        return embedding.cpu().numpy().flatten()

    except Exception as e:
        print(f"❌ ERROR: {filepath} — {e}")
        return None

# Main loop with incremental saving
embeddings = []
filenames = []
failures = []

start_idx = 0  # Change to resume if needed

for i, path in enumerate(tqdm(mp3_files[start_idx:], desc="Embedding audio files"), start=start_idx):
    vec = embed_audio(path)
    if vec is not None:
        embeddings.append(vec)
        filenames.append(path)
    else:
        failures.append(path)

    if (i + 1) % SAVE_EVERY == 0:
        np.save(f"{output_dir}/vectors_{i+1}.npy", np.vstack(embeddings))
        np.save(f"{output_dir}/filenames_{i+1}.npy", np.array(filenames))
        print(f"✅ Saved checkpoint at {i+1} tracks")
        embeddings.clear()
        filenames.clear()

# Final save for leftovers
if embeddings:
    np.save(f"{output_dir}/vectors_final.npy", np.vstack(embeddings))
    np.save(f"{output_dir}/filenames_final.npy", np.array(filenames))
    print("✅ Final save complete")

# Save failures
if failures:
    with open(f"{output_dir}/failures.txt", "w") as f:
        for fail in failures:
            f.write(f"{fail}\n")
    print(f"❌ {len(failures)} failures saved to failures.txt")


Embedding audio files:   1%|             | 570/106574 [01:06<2:57:08,  9.97it/s]

❌ ERROR: /home/ivan/PycharmProjects/MPr/audio_samples/fma_large/fma_large/115/115610.mp3 — Failed to open the input "/home/ivan/PycharmProjects/MPr/audio_samples/fma_large/fma_large/115/115610.mp3" (Invalid argument).


Embedding audio files:   1%|            | 1001/106574 [01:50<2:49:00, 10.41it/s]

✅ Saved checkpoint at 1000 tracks


Embedding audio files:   2%|▏           | 2001/106574 [03:49<3:52:01,  7.51it/s]

...

❌ ERROR: /home/ivan/PycharmProjects/MPr/audio_samples/fma_large/fma_large/148/148792.mp3 — Failed to open the input "/home/ivan/PycharmProjects/MPr/audio_samples/fma_large/fma_large/148/148792.mp3" (Invalid argument).


Embedding audio files: 100%|█████████▉| 106493/106574 [3:04:49<00:06, 12.15it/s]

❌ ERROR: /home/ivan/PycharmProjects/MPr/audio_samples/fma_large/fma_large/148/148789.mp3 — Failed to open the input "/home/ivan/PycharmProjects/MPr/audio_samples/fma_large/fma_large/148/148789.mp3" (Invalid argument).


Embedding audio files: 100%|██████████| 106574/106574 [3:04:58<00:00,  9.60it/s]

✅ Final save complete
❌ 162 failures saved to failures.txt


# Сбор всех чекпоинтов в единый файл

Чтобы потом загрузить один `.npy` вместо множества небольших:
1) Читаем все `vectors_*.npy` и склеиваем их по вертикали.
2) То же для `filenames_*.npy`.
3) Сохраняем итоговые `audio_vectors.npy` и `audio_filenames.npy`.


In [7]:
import os
import numpy as np
from glob import glob

# Paths
output_dir = "/home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large"

# 1. Load & stack all vector files
vector_files = sorted(glob(f"{output_dir}/vectors_*.npy"))
vectors_list = []

print(f"🔍 Found {len(vector_files)} vector chunks to merge")

for vf in vector_files:
    print(f"➕ Loading {vf}")
    vectors_list.append(np.load(vf))

all_vectors = np.vstack(vectors_list)
print(f"✅ Final vectors shape: {all_vectors.shape}")

# 2. Load & stack all filename files
filename_files = sorted(glob(f"{output_dir}/filenames_*.npy"))
filenames_list = []

print(f"🔍 Found {len(filename_files)} filename chunks to merge")

for ff in filename_files:
    print(f"➕ Loading {ff}")
    filenames_list.append(np.load(ff))

all_filenames = np.concatenate(filenames_list)
print(f"✅ Final filenames shape: {all_filenames.shape}")

# 3. Save merged outputs
np.save(f"{output_dir}/audio_vectors.npy", all_vectors)
np.save(f"{output_dir}/audio_filenames.npy", all_filenames)

print(f"✅ Merged files saved to {output_dir}/audio_vectors.npy and audio_filenames.npy")


🔍 Found 107 vector chunks to merge
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_1000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_10000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_100000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_101000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_102000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_103000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_104000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_105000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_106000.npy
➕ Loading /home/ivan/PycharmProjects/MPr/notebooks/embeddings/embedings_large/vectors_11000.npy

In [29]:
import os
import numpy as np

np.save("embeddings/audio_vectors_large.npy", embeddings)
np.save("embeddings/audio_filenames_large.npy", np.array(filenames))


# Проверяем, что итоговые файлы на месте и корректно загружаются

Загружаем `audio_vectors.npy` и `audio_filenames.npy`,  
печатаем форму, чтобы убедиться, что всё собрано как надо.


In [30]:
import os

print("audio_vectors.npy:", os.path.exists("embeddings/audio_vectors_large.npy"))
print("audio_filenames.npy:", os.path.exists("embeddings/audio_filenames_large.npy"))


audio_vectors.npy: True
audio_filenames.npy: True


In [31]:
import numpy as np

vectors = np.load("embeddings/audio_vectors_large.npy")
filenames = np.load("embeddings/audio_filenames_large.npy")

print("✅ Vectors shape:", vectors.shape)      # Should be (N, 512)
print("✅ Filenames shape:", filenames.shape)  # Should be (N,)


✅ Vectors shape: (300, 512)
✅ Filenames shape: (300,)
